In [ ]:
%matplotlib inline

def f(x, y):
    return np.exp(x) * np.exp(y)

# Define the grid
num_points = 100
x_min, x_max = 0, 1
y_min, y_max = -1, 0
x_grid, y_grid = np.meshgrid(np.linspace(x_min, x_max, 100), np.linspace(y_min, y_max, 100))

# Evaluate the function on the grid
z_grid = f(x_grid, y_grid)

# Plot the surface
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(x_grid, y_grid, z_grid)
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('f(x,y)')
plt.show()
plt.close()

In [ ]:
def chebyshev_nodes(p, n):
    """Compute the Chebyshev-Gauss-Lobatto collocation nodes and polinomials in the nodes"""
    # nodes = np.cos( ((2 * (np.arange(n)+1) ) - 1)/ (2 * n) * np.pi )
    nodes = np.cos((2*np.arange(1,n+1) - 1) * np.pi / (2*n))
    # define the chebyshev polinomials
    T_nodes = np.zeros((n,p+1))
    T_nodes[:,0] = np.ones((n,1)).T
    T_nodes[:,1] = nodes.T
    for i in range(1,p):
        T_nodes[:,i+1] = 2 * nodes * T_nodes[:,i] - T_nodes[:,i-1]
    # or numpy's routine
    #T = np.polynomial.chebyshev.chebvander(nodes,p) 
    return nodes, T_nodes

# Compute the sum of squared residuals
def sse(gamma, fun, b, T):
    fun=fun(b) 
    R = fun - T @ gamma
    return np.sum(R**2)

# change variable from b (argument in the actual function) to x (argument in the polinomials)
def b_to_x(b, b_max, b_min):
    return (2*b)/(b_max-b_min)-(b_max+b_min)/(b_max-b_min)

# change variable from x (argument in the polinomials) to b (argument in the actual function)
def x_to_b(x, b_max, b_min):
    return (x+1)*(b_max-b_min)/2 + b_min

def get_gamma(fun, interval, parameters):
    # define the interval in which the function is defined
    interval_min = interval[0]
    interval_max = interval[1]
    # set number of nodes and approximation order 
    n = parameters[0]
    p = parameters[1]
    # compute the nodes and the Chebyshev polinomials in the nodes
    nodes, T_nodes = chebyshev_nodes(p, n)
    # compute weights that minimize the sum of squared residuals
    b = x_to_b(nodes, interval_max, interval_min)
    res = optimize.minimize(sse, x0=np.zeros(p+1), args=(fun, b, T_nodes))
    gamma = res.x
    return gamma    

# compute the grid for the approximation
def approx_grid(b_grid,p,gamma):
    # grid for x
    x_grid_app = b_to_x(b_grid, np.max(b_grid), np.min(b_grid))
    # compute the polinomias value for any point in the grid
    T = np.zeros((len(b_grid),p+1))
    T[:,0] = np.ones((len(b_grid),1)).T
    T[:,1] = x_grid_app.T
    for i in range(1,p):
        T[:,i+1] = 2 * x_grid_app * T[:,i] - T[:,i-1]
    y_grid_app = T @ gamma
    return y_grid_app

def plot_approx(fun, interval, parameters,gamma,grid_points):
    # define the interval in which the function is defined
    interval_min = interval[0]
    interval_max = interval[1]
    # set number of nodes and approximation order 
    n = parameters[0]
    p = parameters[1]
    # grid
    b_grid = np.linspace(interval_min,interval_max,grid_points)
    # grid of actual function 
    y_grid = fun(b_grid)
    # grid of approximated function 
    y_grid_app = approx_grid(b_grid,p,gamma)
    # Plot the surface
    fig = plt.figure()
    plt.plot(b_grid,y_grid, 'k-')
    plt.plot(b_grid,y_grid_app, 'k:')
    plt.show()
    plt.close()

In [ ]:
grid_points=100

def z(b):
    return np.exp(b)
fun=z
interval=[0,2]
parameters=[3,3]
gamma = get_gamma(fun,interval,parameters)
plot_approx(fun,interval,parameters,gamma,grid_points)

def g(x):
    return np.maximum(0, x - 1)
fun=g
interval=[0,2]
parameters=[20,20]
gamma = get_gamma(fun,interval,parameters)
plot_approx(fun,interval,parameters,gamma,grid_points)